In [444]:
import numpy as np
import pandas as pd
import pylab as pl
import geopandas as gpd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
from matplotlib import gridspec
from fiona.crs import from_epsg
import shapely
import choroplethNYC as cp
import open311SNSL
from sodapy import Socrata

pd.set_option('display.max_columns', 500)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [445]:
monthlyUsage = pd.read_csv("July 2018 Monthly LinkNYC Data - Per-AP.csv")

In [446]:
monthlyUsage.head()

,AP Device Name,Boro,CB,PPT ID,Street Address,Latitude,Longitude,SSID(s),Uptime %,Number of Unique Clients,Number of Sessions,Minutes Used,MB Transferred (upload),MB Transferred (download),MB Transferred (total),Daily Avg - Number of Unique Clients,Daily Avg - Number of Sessions,Daily Avg - Minutes Used,Daily Avg - MB Transferred (upload),Daily Avg - MB Transferred (download),Daily Avg - MB Transferred (total)
0,bk-01-108703,bk,1,108703,213 UNION AVENUE,40.706822,-73.950529,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",100.00%,"11,593","36,797","102,960","17,046","142,473","159,519",374,"1,187","3,321",550,"4,596","5,146"
1,bk-01-108949,bk,1,108949,709 BROADWAY,40.701419,-73.942908,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",99.87%,"25,219","75,194","234,063","41,436","327,003","368,439",814,"2,426","7,550","1,337","10,548","11,885"
2,bk-01-108952,bk,1,108952,511 BROADWAY,40.705371,-73.949963,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",100.00%,"23,497","56,722","138,142","27,941","194,260","222,201",758,"1,830","4,456",901,"6,266","7,168"
3,bk-01-109091,bk,1,109091,24 GRAHAM AVENUE,40.701542,-73.942169,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",99.87%,"9,336","31,424","179,954","18,587","246,474","265,060",301,"1,014","5,805",600,"7,951","8,550"
4,bk-01-109252,bk,1,109252,395 BROADWAY,40.707208,-73.954195,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",100.00%,"35,990","107,471","298,471","52,864","359,326","412,189","1,894","5,656","15,709","2,782","18,912","21,694"


In [447]:
policyData = pd.read_csv("PolicyMap Data 2019-04-11 1511UTC.csv")

In [448]:
policyData.head()

,Zip Code Tabulation Area,FIPS Code,Formatted FIPS,Estimated percent of households with a subscription to broadband
0,10314,10314.0,"=""10314""",78.71
1,11234,11234.0,"=""11234""",75.72
2,10312,10312.0,"=""10312""",81.77
3,10306,10306.0,"=""10306""",75.95
4,11430,11430.0,"=""11430""",NaN


In [449]:
shapefileNYC = gpd.GeoDataFrame.from_file("ZIP_CODE_040114/ZIP_CODE_040114.shp")

In [450]:
shapefileNYC.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,..."


In [451]:
shapefileNYC.shape

(263, 13)

In [452]:
shapefileNYC = shapefileNYC.merge(policyData, left_on = "ZIPCODE", right_on = "Zip Code Tabulation Area", how = 'left')

In [453]:
shapefileNYC.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry,Zip Code Tabulation Area,FIPS Code,Formatted FIPS,Estimated percent of households with a subscription to broadband
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,...",11436,11436.0,"=""11436""",65.37
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,...",11213,11213.0,"=""11213""",62.95
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1...",11212,11212.0,"=""11212""",58.12
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,...",11225,11225.0,"=""11225""",67.98
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,...",11218,11218.0,"=""11218""",70.67


In [454]:
shapefileNYC.drop(['Zip Code Tabulation Area', 'FIPS Code', 'Formatted FIPS'], axis = 1, inplace = True)

In [455]:
shapefileNYC.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry,Estimated percent of households with a subscription to broadband
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,...",65.37
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,...",62.95
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1...",58.12
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,...",67.98
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,...",70.67


In [456]:
shapefileNYC_ft = shapefileNYC.to_crs(epsg=2263)

In [457]:
monthlyUsage['lonlat'] = list(zip(monthlyUsage.Longitude,monthlyUsage.Latitude))
monthlyUsage['geometry'] = monthlyUsage[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

In [458]:
monthlyUsage = gpd.GeoDataFrame(monthlyUsage)

In [459]:
monthlyUsage.crs = from_epsg(4326)

In [460]:
monthlyUsage_ft = monthlyUsage.to_crs(epsg=2263)

In [461]:
monthlyUsageZipcode = gpd.sjoin(shapefileNYC_ft, monthlyUsage_ft)

In [462]:
monthlyUsageZipcode.reset_index(drop = True, inplace = True)

In [463]:
monthlyUsageZipcode.head(1)

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry,Estimated percent of households with a subscription to broadband,index_right,AP Device Name,Boro,CB,PPT ID,Street Address,Latitude,Longitude,SSID(s),Uptime %,Number of Unique Clients,Number of Sessions,Minutes Used,MB Transferred (upload),MB Transferred (download),MB Transferred (total),Daily Avg - Number of Unique Clients,Daily Avg - Number of Sessions,Daily Avg - Minutes Used,Daily Avg - MB Transferred (upload),Daily Avg - MB Transferred (download),Daily Avg - MB Transferred (total),lonlat
0,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.71296406 186926.4395293033, ...",62.95,94,bk-03-145749,bk,3,145749,1628 FULTON STREET,40.679561,-73.9355,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",100.00%,"15,182","42,873","111,437","22,290","186,040","208,330",490,"1,383","3,595",719,"6,001","6,720","(-73.93549973, 40.67956068)"


In [464]:
monthlyUsageZipcode.drop(['BLDGZIP', 'CTY_FIPS', 'ST_FIPS', 'SSID(s)', 'STATE', 'COUNTY', 'URL', 'SHAPE_AREA', 'SHAPE_LEN', 'index_right', 'Boro', ], axis = 1, inplace = True)

In [465]:
monthlyUsageZipcode.head()

,ZIPCODE,PO_NAME,POPULATION,AREA,geometry,Estimated percent of households with a subscription to broadband,AP Device Name,CB,PPT ID,Street Address,Latitude,Longitude,Uptime %,Number of Unique Clients,Number of Sessions,Minutes Used,MB Transferred (upload),MB Transferred (download),MB Transferred (total),Daily Avg - Number of Unique Clients,Daily Avg - Number of Sessions,Daily Avg - Minutes Used,Daily Avg - MB Transferred (upload),Daily Avg - MB Transferred (download),Daily Avg - MB Transferred (total),lonlat
0,11213,Brooklyn,62426.0,2.963100e+07,"POLYGON ((1001613.71296406 186926.4395293033, ...",62.95,bk-03-145749,3,145749,1628 FULTON STREET,40.679561,-73.935500,100.00%,"15,182","42,873","111,437","22,290","186,040","208,330",490,"1,383","3,595",719,"6,001","6,720","(-73.93549973, 40.67956068)"
1,11213,Brooklyn,62426.0,2.963100e+07,"POLYGON ((1001613.71296406 186926.4395293033, ...",62.95,bk-03-145759,3,145759,1546 FULTON STREET,40.679752,-73.939101,100.00%,"13,998","54,966","300,095","41,523","297,594","339,118",452,"1,773","9,680","1,339","9,600","10,939","(-73.93910101, 40.67975193)"
2,11225,Brooklyn,56527.0,2.369863e+07,"POLYGON ((995908.3654508907 183617.6128136422,...",67.98,bk-09-127022,9,127022,1208 NOSTRAND AVENUE,40.657740,-73.950407,100.00%,"12,421","58,252","303,969","38,068","424,068","462,136",401,"1,879","9,805","1,228","13,680","14,908","(-73.95040697, 40.65773965)"
3,11225,Brooklyn,56527.0,2.369863e+07,"POLYGON ((995908.3654508907 183617.6128136422,...",67.98,bk-09-127021,9,127021,1106 NOSTRAND AVENUE,40.660732,-73.950727,99.87%,"10,060","35,284","140,386","22,479","181,024","203,503",325,"1,138","4,529",725,"5,839","6,565","(-73.95072664, 40.6607318)"
4,11225,Brooklyn,56527.0,2.369863e+07,"POLYGON ((995908.3654508907 183617.6128136422,...",67.98,bk-09-145844,9,145844,317 LEFFERTS AVENUE,40.662438,-73.950605,99.87%,"9,418","37,671","128,707","14,693","175,966","190,658",304,"1,215","4,152",474,"5,676","6,150","(-73.950605, 40.662438)"


In [466]:
columnName = ['Number of Unique Clients', 'Number of Sessions', 'Minutes Used', 'MB Transferred (upload)', 'MB Transferred (download)', 
              'MB Transferred (total)', 'Daily Avg - Number of Unique Clients', 'Daily Avg - Number of Sessions', 'Daily Avg - Minutes Used', 
              'Daily Avg - MB Transferred (upload)', 'Daily Avg - MB Transferred (download)', 'Daily Avg - MB Transferred (total)']

In [467]:
for name in columnName:
    monthlyUsageZipcode[name] = monthlyUsageZipcode[name].apply(lambda x: x.replace(',', ''))
    monthlyUsageZipcode[name] = monthlyUsageZipcode[name].astype('float') 

In [468]:
monthlyUsageZipcodeMean = monthlyUsageZipcode.groupby('ZIPCODE').mean()[columnName].reset_index()

In [469]:
shapefileUsage = shapefileNYC.merge(monthlyUsageZipcodeMean)

In [470]:
shapefileUsage.head(1)

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry,Estimated percent of households with a subscription to broadband,Number of Unique Clients,Number of Sessions,Minutes Used,MB Transferred (upload),MB Transferred (download),MB Transferred (total),Daily Avg - Number of Unique Clients,Daily Avg - Number of Sessions,Daily Avg - Minutes Used,Daily Avg - MB Transferred (upload),Daily Avg - MB Transferred (download),Daily Avg - MB Transferred (total)
0,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,...",62.95,14590.0,48919.5,205766.0,31906.5,241817.0,273724.0,471.0,1578.0,6637.5,1029.0,7800.5,8829.5


In [471]:
shapefileUsage.corr()['Estimated percent of households with a subscription to broadband']

POPULATION                                                         -0.517715
AREA                                                               -0.260298
SHAPE_AREA                                                               NaN
SHAPE_LEN                                                                NaN
Estimated percent of households with a subscription to broadband    1.000000
Number of Unique Clients                                            0.298728
Number of Sessions                                                  0.234978
Minutes Used                                                        0.042819
MB Transferred (upload)                                             0.153618
MB Transferred (download)                                          -0.147193
MB Transferred (total)                                             -0.107310
Daily Avg - Number of Unique Clients                                0.091086
Daily Avg - Number of Sessions                                     -0.001463

In [472]:
client = Socrata("data.cityofnewyork.us",
                 "jjCh6OTqr3VrUwNTNl4bPGwAK")

results = client.get("fhrw-4uyv", where = "created_date BETWEEN  '2018-07-01T00:00:00' AND '2018-07-31T00:00:00'", limit = 300000)

data311 = pd.DataFrame.from_records(results)

In [473]:
data311.shape

(227494, 41)

In [474]:
data311.dropna

<bound method DataFrame.dropna of         address_type agency                              agency_name  \
0            ADDRESS  DOHMH  Department of Health and Mental Hygiene   
1            ADDRESS  DOHMH  Department of Health and Mental Hygiene   
2            ADDRESS   DSNY                 Department of Sanitation   
3            ADDRESS  DOHMH  Department of Health and Mental Hygiene   
4            ADDRESS  DOHMH  Department of Health and Mental Hygiene   
5            ADDRESS  DOHMH  Department of Health and Mental Hygiene   
6            LATLONG  DOHMH  Department of Health and Mental Hygiene   
7            LATLONG  DOHMH  Department of Health and Mental Hygiene   
8            LATLONG  DOHMH  Department of Health and Mental Hygiene   
9            LATLONG  DOHMH  Department of Health and Mental Hygiene   
10           ADDRESS  DOHMH  Department of Health and Mental Hygiene   
11           ADDRESS  DOHMH  Department of Health and Mental Hygiene   
12           ADDRESS  DOHMH  D

In [475]:
data311['lonlat'] = list(zip(data311.longitude.astype(float),data311.latitude.astype(float)))
data311['geometry'] = data311[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

In [476]:
data311 = gpd.GeoDataFrame(data311)
data311.crs = from_epsg(4326)
data311_ft = data311.to_crs(epsg=2263)

In [477]:
monthlyUsage311 = gpd.sjoin(shapefileNYC_ft, data311_ft).groupby('ZIPCODE').size().to_frame()
monthlyUsage311.reset_index(inplace = True)

In [478]:
monthlyUsage311.rename(columns={monthlyUsage311.columns[1]: "311number" }, inplace=True)
monthlyUsage311.head()

,ZIPCODE,311number
0,00083,106
1,10001,972
2,10002,1425
3,10003,1274
4,10004,155


In [479]:
shapefileUsage = shapefileUsage.merge(monthlyUsage311, left_on = 'ZIPCODE', right_on = 'ZIPCODE', how = 'left')

In [480]:
shapefileUsage.corr()['Estimated percent of households with a subscription to broadband']

POPULATION                                                         -0.517715
AREA                                                               -0.260298
SHAPE_AREA                                                               NaN
SHAPE_LEN                                                                NaN
Estimated percent of households with a subscription to broadband    1.000000
Number of Unique Clients                                            0.298728
Number of Sessions                                                  0.234978
Minutes Used                                                        0.042819
MB Transferred (upload)                                             0.153618
MB Transferred (download)                                          -0.147193
MB Transferred (total)                                             -0.107310
Daily Avg - Number of Unique Clients                                0.091086
Daily Avg - Number of Sessions                                     -0.001463

In [481]:
shapefileUsage.corr()['311number']

POPULATION                                                          0.803051
AREA                                                                0.475821
SHAPE_AREA                                                               NaN
SHAPE_LEN                                                                NaN
Estimated percent of households with a subscription to broadband   -0.565523
Number of Unique Clients                                           -0.386210
Number of Sessions                                                 -0.333803
Minutes Used                                                       -0.160102
MB Transferred (upload)                                            -0.306046
MB Transferred (download)                                          -0.013991
MB Transferred (total)                                             -0.074854
Daily Avg - Number of Unique Clients                               -0.221982
Daily Avg - Number of Sessions                                     -0.094868

In [332]:
# #shapefileUsage.crs = from_epsg(4326)
# monthlyUsage.crs = from_epsg(4326)

In [333]:
# shapefileUsage.plot('Estimated percent of households with a subscription to broadband', figsize=(10,10), cmap="RdBu", legend=True)

# f, (ax1, ax2) =pl.subplots(1, 2, figsize=(15,5))

# seattleShpRent2017.plot('zipcode', cmap="RdBu",categorical=True, legend=True, figsize=(10,10), ax=ax1)
# ax1.set_ylabel("latitude", fontsize=20)
# ax1.set_xlabel("longitude", fontsize=20)

# seattleShpRent2017.plot('area', cmap="RdBu", legend=True, figsize=(10,10), ax=ax2)
# ax2.set_ylabel("latitude", fontsize=20)
# ax2.set_xlabel("longitude", fontsize=20)

In [334]:
# monthlyUsage['logNumberUniqueClinets'] = log(monthlyUsage['Daily Avg - Number of Unique Clients'])

In [335]:
# monthlyUsage.plot('logNumberUniqueClinets', figsize=(10,10), cmap="RdBu", legend=True, edgecolor='black', scheme="Equal_interval", k=10)

In [336]:
# ax = monthlyUsage.plot('logNumberUniqueClinets', figsize=(10,10), cmap="RdBu", legend=True, edgecolor='black', scheme="Equal_interval", k=10)
# shapefileUsage.plot('Estimated percent of households with a subscription to broadband',figsize=(10,15), cmap='Reds', ax=ax, markersize=3)

In [337]:
# fig, ax, leg = shapefileUsage.plot('Estimated percent of households with a subscription to broadband', figsize=(10,10), cmap="RdBu", legend=True, ', edgecolor='black', scheme="Equal_interval", k=10)


# cp.choroplethNYC(pumashplc, column='linkNYCp100p', cmap='bone', edgecolor='black', scheme="Equal_interval", k=10)
# linkNYC.plot(c=linkNYC.date_link_,cmap='Reds',ax=ax, markersize=3)
# pl.title("linkNYC locations Per 100 Person Choropleth in 10 equal intervals")

In [338]:
# shapefileUsage.plot('Estimated percent of households with a subscription to broadband', figsize=(10,10), cmap="RdBu", legend=True)

# f, (ax1, ax2) =pl.subplots(1, 2, figsize=(15,5))

# seattleShpRent2017.plot('zipcode', cmap="RdBu",categorical=True, legend=True, figsize=(10,10), ax=ax1)
# ax1.set_ylabel("latitude", fontsize=20)
# ax1.set_xlabel("longitude", fontsize=20)

# seattleShpRent2017.plot('area', cmap="RdBu", legend=True, figsize=(10,10), ax=ax2)
# ax2.set_ylabel("latitude", fontsize=20)
# ax2.set_xlabel("longitude", fontsize=20)

In [339]:
# monthlyUsage['Daily Avg - Number of Unique Clients'] = monthlyUsage['Daily Avg - Number of Unique Clients'].apply(lambda x: x.replace(',', ''))
# monthlyUsage['Daily Avg - Number of Unique Clients'] = monthlyUsage['Daily Avg - Number of Unique Clients'].astype('float') 

In [340]:
#monthlyUsage.plot('Daily Avg - Number of Unique Clients')

#fig, ax, leg = cp.choroplethNYC(monthlyUsage, column='Daily Avg - Number of Unique Clients', cmap='bone', edgecolor='black', scheme="Equal_interval", k=5)
#shapefileUsage.plot('Estimated percent of households with a subscription to broadband',cmap='Reds',ax=ax, markersize=3)
#pl.title("linkNYC locations of Broadband Access percentage Choropleth in 5 equal intervals")

In [341]:
# fig, ax, leg = monthlyUsage.plot('Daily Avg - Number of Unique Clients', figsize=(10,10), cmap="RdBu", legend=True, edgecolor='black', scheme="Equal_interval", k=10)
# shapefileUsage.plot('Estimated percent of households with a subscription to broadband',cmap='Reds',ax=ax, markersize=3)


In [342]:
# shapefileUsage.plot('Estimated percent of households with a subscription to broadband')